In [1]:
!pip install supervision
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.4/212.4 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.7 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
"""
🎯 Script: Dự đoán đối tượng từ video bằng YOLO (skip-frame repeat 5)
----------------------------------------------------------------------
- Sử dụng mô hình YOLO để phát hiện đối tượng (nhãn 'pitch') trong video.
- Chỉ chạy YOLO mỗi 5 khung hình để tăng tốc độ, 4 khung hình giữa dùng lại kết quả trước đó.
- Vẽ kết quả lên video đầu ra.
- Lưu thông tin dự đoán (bounding boxes, confidence, keypoints) vào file JSON.

📦 Yêu cầu:
- Python 3.8+
- Thư viện: ultralytics, opencv-python, numpy
"""

from ultralytics import YOLO
import cv2
import numpy as np
import json

# ============================
# 🚀 Khởi tạo mô hình và video
# ============================

# Tải mô hình YOLO đã huấn luyện
model_pitch = YOLO("/content/drive/MyDrive/Thử nghiệm tối ưu/best_n.pt")

# Đường dẫn video đầu vào
video_path = "/content/drive/MyDrive/Thử nghiệm tối ưu/Test30s.mp4"
cap = cv2.VideoCapture(video_path)

# Thiết lập video đầu ra
save_output = True
if save_output:
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    out = cv2.VideoWriter("output_repeat5.mp4", fourcc, fps, (width, height))

# ============================
# 📦 Lưu dự đoán
# ============================
predictions = []

# ====================================
# 🎨 Hàm vẽ bounding box và keypoints
# ====================================
def draw_detections(frame, boxes, labels, confs, keypoints=None):
    """
    Vẽ bounding box, nhãn và keypoints lên khung hình.

    Args:
        frame (np.ndarray): Khung hình gốc.
        boxes (list): Danh sách bounding box [x1, y1, x2, y2].
        labels (list): Danh sách nhãn tương ứng.
        confs (list): Danh sách độ tin cậy.
        keypoints (list, optional): Danh sách tọa độ keypoints.

    Returns:
        np.ndarray: Khung hình đã vẽ.
    """
    for (box, label, conf) in zip(boxes, labels, confs):
        if label != "pitch":
            continue
        x1, y1, x2, y2 = map(int, box)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"{label} {conf:.2f}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    if keypoints:
        for idx, (x, y) in enumerate(keypoints):
            if x > 0 and y > 0:
                cv2.circle(frame, (int(x), int(y)), 5, (0, 255, 0), -1)
                cv2.putText(frame, str(idx + 1), (int(x) + 5, int(y) - 5),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    return frame

# ==========================================
# 🔍 Dự đoán từng khung hình (skip-frame 5)
# ==========================================
frame_idx = 0
prev_boxes, prev_labels, prev_confs, prev_kps = None, None, None, None

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # ==== CASE 1: Frame chạy YOLO (mỗi 5 khung hình) ====
    if frame_idx % 5 == 0:
        results = model_pitch(frame)[0]

        cur_boxes = [b.xyxy[0].cpu().numpy().tolist() for b in results.boxes]
        cur_labels = [results.names[int(b.cls[0])] for b in results.boxes]
        cur_confs = [float(b.conf[0]) for b in results.boxes]

        # Trích xuất keypoints nếu có
        if hasattr(results, "keypoints") and results.keypoints is not None and len(results.keypoints.xy) > 0:
            cur_kps = results.keypoints.xy[0].cpu().numpy().tolist()
        else:
            cur_kps = []

        # Lưu dự đoán JSON
        predictions.append({
            "frame_id": frame_idx,
            "pitch": {
                "boxes": [
                    {"bbox": box, "class": label, "conf": conf}
                    for box, label, conf in zip(cur_boxes, cur_labels, cur_confs)
                    if label == "pitch"
                ],
                "keypoints": cur_kps
            }
        })

        # Vẽ & ghi video
        final_frame = draw_detections(frame, cur_boxes, cur_labels, cur_confs, cur_kps)
        if save_output:
            out.write(final_frame)

        # Cập nhật bộ nhớ cho 4 khung hình tiếp theo
        prev_boxes, prev_labels, prev_confs, prev_kps = cur_boxes, cur_labels, cur_confs, cur_kps

    # ==== CASE 2: 4 frame giữa → dùng lại kết quả trước đó ====
    else:
        if prev_boxes is not None:
            predictions.append({
                "frame_id": frame_idx,
                "pitch": {
                    "boxes": [
                        {"bbox": box, "class": label, "conf": conf}
                        for box, label, conf in zip(prev_boxes, prev_labels, prev_confs)
                        if label == "pitch"
                    ],
                    "keypoints": prev_kps if prev_kps else []
                }
            })

            # Vẽ khung hình hiện tại bằng kết quả trước đó
            frame_drawn = draw_detections(frame, prev_boxes, prev_labels, prev_confs, prev_kps)
            if save_output:
                out.write(frame_drawn)

    frame_idx += 1

# ============================
# 💾 Cleanup & lưu kết quả
# ============================
json_file = "predictions_repeat5.json"
with open(json_file, "w") as f:
    json.dump(predictions, f, indent=4)

cap.release()
if save_output:
    out.release()

print(f"✅ Đã xử lý xong {frame_idx} khung hình. Video lưu thành 'output_repeat5.mp4', JSON lưu thành '{json_file}'.")


0: 384x640 1 pitch, 13.8ms
Speed: 1.8ms preprocess, 13.8ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pitch, 8.7ms
Speed: 2.1ms preprocess, 8.7ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pitch, 9.2ms
Speed: 1.7ms preprocess, 9.2ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pitch, 8.9ms
Speed: 2.3ms preprocess, 8.9ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pitch, 10.1ms
Speed: 2.2ms preprocess, 10.1ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pitch, 10.0ms
Speed: 2.2ms preprocess, 10.0ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pitch, 10.0ms
Speed: 3.0ms preprocess, 10.0ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pitch, 9.2ms
Speed: 2.4ms preprocess, 9.2ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x